# Loading Plots to IScience Portal


To remotely upload plots, we will use the REST API provided by the iScience server.  There are several steps involved with uploading then connecting the upload with an experiment. The relationship between these is described as:

Experiment -- [1-many]-- ExperimentResult -- [1 to 1] -- Plot

The steps are as follows:

1. Create a Session (username/password) to get a token
1. Create Plot and save to HTML file
1. Create or Select an Experiment
2. Create an Experiment Result containing plot file and link it to the experiment

In [1]:
! pip install plotly==4.2.1
! conda install -y -c plotly plotly==4.2.1
! conda install -y -c conda-forge cufflinks-py

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - plotly==4.2.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    plotly-4.2.1               |             py_0         4.1 MB  plotly
    retrying-1.3.3             |             py_2          11 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

  plotly             plotly/

## Create a plot directory

In [ ]:
import os
DATA_DIR = os.path.join(os.getcwd(), 'data')
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)
print(DATA_DIR)

## Create a plotly plot


In [2]:
import plotly
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd


df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

print(df.head())

ModuleNotFoundError: No module named 'plotly'

In [18]:
# Create a sample graph as figure and save to data directory
import plotly.graph_objects as go
data = [go.Bar(x=df.School,
            y=df.Gap)]
fig = go.Figure(
    data=data,
    layout_title_text="Sample Figure Bar graph"
)
fig.show()
plot_filename = os.path.join(DATA_DIR, 'plotly_plot.html')
fig.write_html(plot_filename)


## Upload to IScience Portal

### Create an authenticated session

In [19]:
username=input('What is your username? ')

What is your username? demo


In [20]:
password=input('What is your password (this will show in plain text)? ')

What is your password (this will show in plain text)? iScience2019


In [21]:
import requests
from requests.auth import HTTPBasicAuth
import json
from urllib.parse import urlparse, urljoin
import petl as etl
import certifi
import urllib3
# Using SSL so require certification but no access to certs so turn off the warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Enter URL for experiments API
exptbaseurl = 'https://agri-ml.it.csiro.au:8443'

# Get an access token - replace user and password with actual values
serviceurl = exptbaseurl + '/api-token-auth/'
data = dict(username=username, password=password)

auth = requests.post(serviceurl, verify=False, data=data)

if auth.status_code != 200:
    print("Could not successfully access REST API: " + serviceurl)
else:
    token = 'Token ' + auth.json()['token']
    print(token)

Token 37a561a3c66eb68eb95e8101905ce49a087be3ef


### Upload Plot URL

1. Find an experiment to link to in IScience Portal and get ID
OR
1. Create an experiment
1. Create Experiment Result with plot filename


#### Attach to existing Experiment
If an experiment has already been set up in the iScience portal, providing it's ID can allow you to link the VR media to the Experiment Results, as follows.

In [23]:
# Get Experiment ID (lookup in IScience Portal)
expt_id = 10

# Get URL and header
expt_results_url = exptbaseurl + '/experiment_results/'
headers = {'Authorization': token}

# Set Experiment Result details with plot width/height for display
data = {"id": "0", 
        "description": "Basic Bar Chart in Plotly", 
        "result_type": "PLT",
        "experiment": str(expt_id), 
        "width": '100%',
        "height": "480px"
       }

# Add plot file
files = {"plotfile" : open(plot_filename, "rb"),
        "thumbnail": open(os.path.join(DATA_DIR, "Plot.PNG"), "rb")}

# Send to the experiments server
try:
    r = requests.post(expt_results_url, verify=False, headers=headers, files=files, data=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully uploaded Plot to Experiment: id = " + str(response["id"]))
    expt_result_id = response["id"] # the new ID assigned to the Experiment Result
except requests.exceptions.HTTPError as e:
    print(e)   

Successfully uploaded Plot to Experiment: id = 30


#### Check the Plot (Experiment Result) in the database

In [24]:
headers = {'Authorization': token, 'Content-type': 'application/json'}
url = expt_results_url + str(expt_result_id)
try:
    r = requests.get(url, verify=False, headers=headers)
    r.raise_for_status()
    response = r.json()
    print("Content of Experimental Result: id = " + str(expt_result_id))
    print("***********")
    for key in response.keys():
        print(key + ": " + str(response[key]))
except requests.exceptions.HTTPError as e:
    print(e) 

Content of Experimental Result: id = 30
***********
id: 30
description: Basic Bar Chart in Plotly
created: 2020-02-27T01:33:43.929361Z
result_type: PLT
thumbnail: https://agri-ml.it.csiro.au:8443/media/Plot.PNG
imagefile: None
videofile: None
width: 100%
height: 480px
plotfile: https://agri-ml.it.csiro.au:8443/media/plotly_plot_z8DR5U6.html
experiment: 10
vrmodule: None


### OR
#### Create a new Experiment to link to
If you need to create the experiment first, there are some more data to provide.

*It is important to ensure the type and status match the expected values as they are lookup values*

In [12]:
data = {
    "type": {
        "id": 2,
        "name": "Spectroscopy"
    },
    "status": {
        "id": 1,
        "name": "Active"
    },
    "name": "Experiment for uploaded VR",
    "number": "001",
    "accession": "accessnumber",
    "comments": "comments here",
    "samplecode": "",
    "summary": "CREATED from Jupyter Notebook",
    "uident": "tst001",
    "results": []
}

Otherwise, it is very similar to the POST request above

In [13]:
expt_url = exptbaseurl + '/experiments/'
headers = {'Authorization': token, 'Content-type': 'application/json'}
try:
    r = requests.post(expt_url, verify=False, headers=headers, json=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully created the new Experiment: id=" + str(response["id"]))
    expt_id = response["id"]
except requests.exceptions.HTTPError as e:
    print(e)  

Successfully created the new Experiment: id=12


#### Add plot to new experiment
And then use the experiment ID as in the previous section, to link the plot to this experiment.

In [14]:
# Get URL and header
expt_results_url = exptbaseurl + '/experiment_results/'
headers = {'Authorization': token}

# Set Experiment Result details with plot width/height for display
data = {"id": "0", 
        "description": "Basic Bar Chart in Plotly (from notebook)", 
        "result_type": "PLT",
        "experiment": str(expt_id), 
        "width": '100%',
        "height": "480px"
       }

# Add plot file
files = {"plotfile" : open(plot_filename, "rb")}

# Send to the experiments server
try:
    r = requests.post(expt_results_url, verify=False, headers=headers, files=files, data=data)
    r.raise_for_status()
    response = r.json()
    print("Successfully uploaded Plot to Experiment: id = " + str(response["id"]))
    expt_result_id = response["id"] # the new ID assigned to the Experiment Result
except requests.exceptions.HTTPError as e:
    print(e)   

Successfully uploaded Plot to Experiment: id = 28


**When all done, login to the portal at https://agri-ml.it.csiro.au/ and view your new results**